# NBA Predictor: Basketball Advanced Regression, Kyle’s Loss Estimate Yielder (BARKLEY)

## Importing Modules

In [254]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
from sklearn.impute import SimpleImputer

## Loading Data

In [134]:
# 2018 season will be used for testing
games2018 = pd.read_csv('data/games_2018_season.csv')
stats2018 = pd.read_csv('data/stats_2018_season.csv')

In [281]:
df_games = pd.concat(
    [pd.read_csv('data/games_2010_season.csv'),
     pd.read_csv('data/games_2011_season.csv'),
     pd.read_csv('data/games_2012_season.csv'),
     pd.read_csv('data/games_2013_season.csv'),
     pd.read_csv('data/games_2014_season.csv'),
     pd.read_csv('data/games_2015_season.csv'),
     pd.read_csv('data/games_2016_season.csv'),
     pd.read_csv('data/games_2017_season.csv')]
)

In [230]:
df_stats = pd.concat(
    [pd.read_csv('data/stats_2010_season.csv'),
     pd.read_csv('data/stats_2011_season.csv'),
     pd.read_csv('data/stats_2012_season.csv'),
     pd.read_csv('data/stats_2013_season.csv'),
     pd.read_csv('data/stats_2014_season.csv'),
     pd.read_csv('data/stats_2015_season.csv'),
     pd.read_csv('data/stats_2016_season.csv'),
     pd.read_csv('data/stats_2017_season.csv')]
)

In [196]:
df_games.head()

,id,date,home_team_score,period,postseason,season,status,time,visitor_team_score,home_team.id,home_team.abbreviation,home_team.city,home_team.conference,home_team.division,home_team.full_name,home_team.name,visitor_team.id,visitor_team.abbreviation,visitor_team.city,visitor_team.conference,visitor_team.division,visitor_team.full_name,visitor_team.name
0,24362,2010-10-27T00:00:00.000Z,95,4,False,2010,Final,,87,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
1,24363,2010-10-27T00:00:00.000Z,104,4,False,2010,Final,,119,15,MEM,Memphis,West,Southwest,Memphis Grizzlies,Grizzlies,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
2,24364,2010-10-27T00:00:00.000Z,106,4,False,2010,Final,,95,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder,5,CHI,Chicago,East,Central,Chicago Bulls,Bulls
3,24365,2010-10-27T00:00:00.000Z,122,4,False,2010,Final,,109,27,SAS,San Antonio,West,Southwest,San Antonio Spurs,Spurs,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
4,24366,2010-10-29T00:00:00.000Z,101,4,False,2010,Final,,95,19,NOP,New Orleans,West,Southwest,New Orleans Pelicans,Pelicans,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets


In [197]:
df_games.columns

Index(['id', 'date', 'home_team_score', 'period', 'postseason', 'season',
       'status', 'time', 'visitor_team_score', 'home_team.id',
       'home_team.abbreviation', 'home_team.city', 'home_team.conference',
       'home_team.division', 'home_team.full_name', 'home_team.name',
       'visitor_team.id', 'visitor_team.abbreviation', 'visitor_team.city',
       'visitor_team.conference', 'visitor_team.division',
       'visitor_team.full_name', 'visitor_team.name'],
      dtype='object')

In [262]:
games_columns = ['id', 'date', 'home_team_score', 'postseason', 'season', 'visitor_team_score', 'home_team.id',
       'home_team.abbreviation', 'home_team.conference',
       'home_team.division',
       'visitor_team.id', 'visitor_team.abbreviation',
       'visitor_team.conference', 'visitor_team.division']

In [282]:
#filter columns and sort by date
df_games = df_games.filter(games_columns).sort_values('date')

In [283]:
df_games.head()

,id,date,home_team_score,postseason,season,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division
1197,27018,2010-10-26T00:00:00.000Z,106,False,2010,92,25,POR,West,Northwest,24,PHX,West,Pacific
992,26553,2010-10-26T00:00:00.000Z,88,False,2010,80,2,BOS,East,Atlantic,16,MIA,East,Southeast
993,26554,2010-10-26T00:00:00.000Z,112,False,2010,110,14,LAL,West,Pacific,11,HOU,West,Southwest
0,24362,2010-10-27T00:00:00.000Z,95,False,2010,87,6,CLE,East,Central,2,BOS,East,Atlantic
1199,27020,2010-10-27T00:00:00.000Z,116,False,2010,117,18,MIN,West,Northwest,26,SAC,West,Pacific


In [284]:
df_stats.head()

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.period,game.postseason,game.season,game.status,game.time,game.visitor_team_id,game.visitor_team_score,player.id,player.first_name,player.height_feet,player.height_inches,player.last_name,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,581359,3.0,0.0,2.0,0.0,0.0,0.0,0.750,4.0,3.0,0.50,2.0,1.0,21:02,0.0,4.0,7.0,2.0,0.0,2.0,24362,2010-10-27T00:00:00.000Z,6,95,4,False,2010,Final,,2,87,735,Shaquille,NaN,NaN,O'Neal,NaN,22,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
1,581358,1.0,1.0,14.0,0.0,0.0,0.0,0.375,8.0,3.0,0.75,4.0,3.0,29:08,1.0,3.0,9.0,15.0,2.0,3.0,24362,2010-10-27T00:00:00.000Z,6,95,4,False,2010,Final,,2,87,971,Kevin,NaN,NaN,Garnett,NaN,18,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
2,581360,4.0,0.0,2.0,0.0,5.0,0.0,0.308,13.0,4.0,0.80,5.0,4.0,35:26,2.0,3.0,12.0,4.0,0.0,1.0,24362,2010-10-27T00:00:00.000Z,6,95,4,False,2010,Final,,2,87,1036,Ray,NaN,NaN,Allen,G,17,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
3,581364,0.0,0.0,2.0,0.0,0.0,0.0,0.500,2.0,1.0,0.00,0.0,0.0,12:22,0.0,6.0,2.0,2.0,0.0,3.0,24362,2010-10-27T00:00:00.000Z,6,95,4,False,2010,Final,,2,87,1059,Jermaine,NaN,NaN,O'Neal,NaN,25,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
4,581372,6.0,0.0,2.0,0.5,4.0,2.0,0.333,12.0,4.0,0.00,2.0,0.0,34:54,0.0,1.0,10.0,2.0,0.0,2.0,24362,2010-10-27T00:00:00.000Z,6,95,4,False,2010,Final,,2,87,1139,Anthony,NaN,NaN,Parker,NaN,23,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers


In [285]:
df_stats.columns

Index(['id', 'ast', 'blk', 'dreb', 'fg3_pct', 'fg3a', 'fg3m', 'fg_pct', 'fga',
       'fgm', 'ft_pct', 'fta', 'ftm', 'min', 'oreb', 'pf', 'pts', 'reb', 'stl',
       'turnover', 'game.id', 'game.date', 'game.home_team_id',
       'game.home_team_score', 'game.period', 'game.postseason', 'game.season',
       'game.status', 'game.time', 'game.visitor_team_id',
       'game.visitor_team_score', 'player.id', 'player.first_name',
       'player.height_feet', 'player.height_inches', 'player.last_name',
       'player.position', 'player.team_id', 'player.weight_pounds', 'team.id',
       'team.abbreviation', 'team.city', 'team.conference', 'team.division',
       'team.full_name', 'team.name'],
      dtype='object')

## Feature Engineering

### Winner

In [286]:
# 1 is home team win, 0 visitor win
df_games['winner'] = np.where(df_games['home_team_score'] > df_games['visitor_team_score'], 1, 0)

### Point Differential

In [287]:
# calculate point differential for the current game for each team
df_games['home_team_diff'] = df_games['home_team_score'] - df_games['visitor_team_score']
df_games['visitor_team_diff'] = df_games['visitor_team_score'] - df_games['home_team_score']

# calculate average point differential for each team for the season (window size 82)
df_games['home_team_avg_diff'] = df_games.groupby(['season', 'home_team.id'])['home_team_diff'].transform(lambda x: x.rolling(82, 1).mean().shift())
df_games['visitor_team_avg_diff'] = df_games.groupby(['season', 'visitor_team.id'])['visitor_team_diff'].transform(lambda x: x.rolling(82, 1).mean().shift())

#last 10 games
df_games['home_team_avg_diff_L10'] = df_games.groupby(['season', 'home_team.id'])['home_team_diff'].transform(lambda x: x.rolling(10, 1).mean().shift())
df_games['visitor_team_avg_diff_L10'] = df_games.groupby(['season', 'visitor_team.id'])['visitor_team_diff'].transform(lambda x: x.rolling(10, 1).mean().shift())

# vs same opponent same team last 8 games (not limited by season)
df_games['home_team_avg_diff_L8_H2H'] = df_games.groupby(['home_team.id','visitor_team.id'])['home_team_diff'].transform(lambda x: x.rolling(8, 1).mean().shift())
df_games['visitor_team_avg_diff_L8_H2H'] = df_games.groupby(['visitor_team.id','home_team.id'])['visitor_team_diff'].transform(lambda x: x.rolling(8, 1).mean().shift())


### Average score

In [288]:
# group by season and team (home or away) and calculates a rolling mean of last x number of games with period of 1. Shift() to exclude the current row to only look at past rows

df_games['home_team_avg_home_score_season'] = df_games.groupby(['season', 'home_team.id'])['home_team_score'].transform(lambda x: x.rolling(82, 1).mean().shift())
df_games['visitor_team_avg_away_score_season'] = df_games.groupby(['season', 'visitor_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(82, 1).mean().shift())

# last 10 games
df_games['home_team_avg_home_score_L10'] = df_games.groupby(['season', 'home_team.id'])['home_team_score'].transform(lambda x: x.rolling(10, 1).mean().shift())
df_games['visitor_team_avg_away_score_L10'] = df_games.groupby(['season', 'visitor_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(10, 1).mean().shift())

# vs same opponent same team last 8 games (not limited by season)
df_games['home_team_avg_home_score_L8_H2H'] = df_games.groupby(['home_team.id','visitor_team.id'])['home_team_score'].transform(lambda x: x.rolling(8, 1).mean().shift())
df_games['visitor_team_avg_away_score_L8_H2H'] = df_games.groupby(['visitor_team.id','home_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(8, 1).mean().shift())


### Average Opponent Score

In [289]:
# for season
df_games['home_team_avg_home_opp_score_season'] = df_games.groupby(['season', 'home_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(82, 1).mean().shift())
df_games['visitor_team_avg_away_opp_score_season'] = df_games.groupby(['season', 'visitor_team.id'])['home_team_score'].transform(lambda x: x.rolling(82, 1).mean().shift())

# last 10
df_games['home_team_avg_home_opp_score_L10'] = df_games.groupby(['season', 'home_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(10, 1).mean().shift())
df_games['visitor_team_avg_away_opp_score_L10'] = df_games.groupby(['season', 'visitor_team.id'])['home_team_score'].transform(lambda x: x.rolling(10, 1).mean().shift())

# vs same opponent same team last 8 games (not limited by season)
df_games['home_team_avg_home_opp_score_L8_H2H'] = df_games.groupby(['home_team.id','visitor_team.id'])['visitor_team_score'].transform(lambda x: x.rolling(8, 1).mean().shift())
df_games['visitor_team_avg_away_opp_score_L8_H2H'] = df_games.groupby(['visitor_team.id','home_team.id'])['home_team_score'].transform(lambda x: x.rolling(8, 1).mean().shift())


In [290]:
df_games[df_games['season'] == 2015]

,id,date,home_team_score,postseason,season,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,winner,home_team_diff,visitor_team_diff,home_team_avg_diff,visitor_team_avg_diff,home_team_avg_diff_L10,visitor_team_avg_diff_L10,home_team_avg_diff_L8_H2H,visitor_team_avg_diff_L8_H2H,home_team_avg_home_score_season,visitor_team_avg_away_score_season,home_team_avg_home_score_L10,visitor_team_avg_away_score_L10,home_team_avg_home_score_L8_H2H,visitor_team_avg_away_score_L8_H2H,home_team_avg_home_opp_score_season,visitor_team_avg_away_opp_score_season,home_team_avg_home_opp_score_L10,visitor_team_avg_away_opp_score_L10,home_team_avg_home_opp_score_L8_H2H,visitor_team_avg_away_opp_score_L8_H2H
877,31090,2015-10-27T00:00:00.000Z,97,False,2015,95,5,CHI,East,Central,6,CLE,East,Central,1,2,-2,NaN,NaN,NaN,NaN,2.125,-2.125,NaN,NaN,NaN,NaN,96.375,94.250,NaN,NaN,NaN,NaN,94.250,96.375
102,27683,2015-10-27T00:00:00.000Z,111,False,2015,95,10,GSW,West,Pacific,19,NOP,West,Southwest,1,16,-16,NaN,NaN,NaN,NaN,10.750,-10.750,NaN,NaN,NaN,NaN,101.625,90.875,NaN,NaN,NaN,NaN,90.875,101.625
101,27682,2015-10-27T00:00:00.000Z,94,False,2015,106,1,ATL,East,Southeast,9,DET,East,Central,0,-12,12,NaN,NaN,NaN,NaN,9.250,-9.250,NaN,NaN,NaN,NaN,103.375,94.125,NaN,NaN,NaN,NaN,94.125,103.375
880,31093,2015-10-28T00:00:00.000Z,76,False,2015,106,15,MEM,West,Southwest,6,CLE,East,Central,0,-30,30,NaN,-2.000000,NaN,-2.0,2.600,-2.600,NaN,95.000000,NaN,95.0,100.800,98.200,NaN,97.000000,NaN,97.0,98.200,100.800
103,27684,2015-10-28T00:00:00.000Z,112,False,2015,95,2,BOS,East,Atlantic,23,PHI,East,Atlantic,1,17,-17,NaN,NaN,NaN,NaN,6.625,-6.625,NaN,NaN,NaN,NaN,98.625,92.000,NaN,NaN,NaN,NaN,92.000,98.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,33917,2016-06-08T00:00:00.000Z,120,True,2015,90,6,CLE,East,Central,10,GSW,West,Pacific,1,30,-30,10.083333,5.583333,16.8,-1.0,-9.250,9.250,107.312500,112.875000,111.0,105.9,97.125,106.375,97.229167,107.291667,94.2,106.9,106.375,97.125
1299,33918,2016-06-10T00:00:00.000Z,97,True,2015,108,6,CLE,East,Central,10,GSW,West,Pacific,0,-11,11,10.489796,4.857143,18.9,-4.7,-4.250,4.250,107.571429,112.408163,111.8,104.6,100.250,104.500,97.081633,107.551020,92.9,109.3,104.500,100.250
1300,33919,2016-06-13T00:00:00.000Z,97,True,2015,112,10,GSW,West,Pacific,6,CLE,East,Central,0,-15,15,14.547170,2.740000,14.6,-2.0,10.250,-10.250,115.150943,101.340000,111.7,98.6,101.750,91.500,100.603774,98.600000,97.1,100.6,91.500,101.750
1301,33920,2016-06-16T00:00:00.000Z,115,True,2015,101,6,CLE,East,Central,10,GSW,West,Pacific,1,14,-14,10.060000,4.980000,16.3,-3.7,-4.000,4.000,107.360000,112.320000,110.6,105.4,100.500,104.500,97.300000,107.340000,94.3,109.1,104.500,100.500


## Train model

In [291]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [292]:
# create X, y
y = df_games['winner']
X = df_games.select_dtypes(include=numerics)

In [319]:
X.drop(['id','season','home_team.id','visitor_team.id','winner','home_team_score','visitor_team_score','home_team_diff','visitor_team_diff'], axis=1,inplace=True)

In [294]:
X=X.apply(lambda x: x.fillna(x.median()),axis=0)

In [330]:
X

,home_team_avg_diff,visitor_team_avg_diff,home_team_avg_diff_L10,visitor_team_avg_diff_L10,home_team_avg_diff_L8_H2H,visitor_team_avg_diff_L8_H2H,home_team_avg_home_score_season,visitor_team_avg_away_score_season,home_team_avg_home_score_L10,visitor_team_avg_away_score_L10,home_team_avg_home_score_L8_H2H,visitor_team_avg_away_score_L8_H2H,home_team_avg_home_opp_score_season,visitor_team_avg_away_opp_score_season,home_team_avg_home_opp_score_L10,visitor_team_avg_away_opp_score_L10,home_team_avg_home_opp_score_L8_H2H,visitor_team_avg_away_opp_score_L8_H2H
1197,3.121324,-2.500000,3.3,-2.7,3.125,-3.125,102.278424,99.345342,102.5,99.8,100.775,97.875,99.000000,101.974359,99.5,102.3,97.875,100.775
992,3.121324,-2.500000,3.3,-2.7,3.125,-3.125,102.278424,99.345342,102.5,99.8,100.775,97.875,99.000000,101.974359,99.5,102.3,97.875,100.775
993,3.121324,-2.500000,3.3,-2.7,3.125,-3.125,102.278424,99.345342,102.5,99.8,100.775,97.875,99.000000,101.974359,99.5,102.3,97.875,100.775
0,3.121324,-2.500000,3.3,-2.7,3.125,-3.125,102.278424,99.345342,102.5,99.8,100.775,97.875,99.000000,101.974359,99.5,102.3,97.875,100.775
1199,3.121324,-2.500000,3.3,-2.7,3.125,-3.125,102.278424,99.345342,102.5,99.8,100.775,97.875,99.000000,101.974359,99.5,102.3,97.875,100.775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,9.120000,3.625000,6.4,-4.9,-5.000,5.000,113.580000,112.541667,109.1,103.8,108.625,113.625,104.460000,108.916667,102.7,108.7,113.625,108.625
1284,9.140000,-0.480000,14.1,-4.2,13.250,-13.250,113.120000,107.800000,113.8,99.2,111.875,98.625,103.980000,108.280000,99.7,103.4,98.625,111.875
1285,9.156863,-0.666667,15.7,-6.6,10.375,-10.375,113.333333,107.921569,114.2,98.3,113.625,103.250,104.176471,108.588235,98.5,104.9,103.250,113.625
1286,2.460000,3.734694,6.6,-2.0,0.750,-0.750,110.440000,112.306122,105.0,103.3,112.125,111.375,107.980000,108.571429,98.4,105.3,111.375,112.125


In [350]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [364]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.80,random_state=1)

In [376]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

In [377]:
# Train the model using the training sets
model.fit(X_train,y_train)


GaussianNB()

In [378]:
y_pred=model.predict(X_test)

In [379]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
print(f'MSE is: {mean_squared_error(y_test, y_pred)}')

#R2 Score
from sklearn.metrics import r2_score
print(f'R2 score is: {r2_score(y_test, y_pred)}')

from sklearn.metrics import accuracy_score
print(f'Accuracy score is: {accuracy_score(y_test, y_pred)}')

from sklearn.metrics import roc_auc_score
print(f'ROC AUC score is: {roc_auc_score(y_test, y_pred)}')

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

MSE is: 0.3568646543330088
R2 score is: -0.48431473842780637
Accuracy score is: 0.6431353456669913
ROC AUC score is: 0.6268406274893328


array([[449, 377],
       [356, 872]])

In [380]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=40, max_depth=6, random_state=0)
clf.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, n_estimators=40, random_state=0)

In [381]:
y_pred=clf.predict(X_test)

In [382]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
print(f'MSE is: {mean_squared_error(y_test, y_pred)}')

#R2 Score
from sklearn.metrics import r2_score
print(f'R2 score is: {r2_score(y_test, y_pred)}')

from sklearn.metrics import accuracy_score
print(f'Accuracy score is: {accuracy_score(y_test, y_pred)}')

from sklearn.metrics import roc_auc_score
print(f'ROC AUC score is: {roc_auc_score(y_test, y_pred)}')

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

MSE is: 0.3485881207400195
R2 score is: -0.44988997641788453
Accuracy score is: 0.6514118792599806
ROC AUC score is: 0.6111721257817984


array([[ 335,  491],
       [ 225, 1003]])

In [385]:
from sklearn.svm import SVC

In [399]:
clf = SVC(kernel='rbf')
clf.fit(X_train,y_train)

SVC()

In [400]:
y_pred=clf.predict(X_test)

In [401]:
#Mean Squared Error
from sklearn.metrics import mean_squared_error
print(f'MSE is: {mean_squared_error(y_test, y_pred)}')

#R2 Score
from sklearn.metrics import r2_score
print(f'R2 score is: {r2_score(y_test, y_pred)}')

from sklearn.metrics import accuracy_score
print(f'Accuracy score is: {accuracy_score(y_test, y_pred)}')

from sklearn.metrics import roc_auc_score
print(f'ROC AUC score is: {roc_auc_score(y_test, y_pred)}')

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

MSE is: 0.3393378773125609
R2 score is: -0.4114152424067954
Accuracy score is: 0.6606621226874392
ROC AUC score is: 0.6135579418097499


array([[ 308,  518],
       [ 179, 1049]])